In [2]:
import deepchem
import deepchem.molnet
from deepchem.data import DiskDataset
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
import warnings
warnings.filterwarnings("ignore")

from tqdm import tqdm
from typing import Literal, Callable, Any, Tuple
import random
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit.Chem import AllChem, MACCSkeys
from rdkit import DataStructs
from sklearn.metrics import roc_auc_score
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, LlamaTokenizer, LlamaForCausalLM

def set_seed(seed):
    torch.manual_seed(seed)  # 设置CPU的随机种子
    torch.cuda.manual_seed(seed)  # 设置当前GPU的随机种子
    torch.cuda.manual_seed_all(seed)  # 设置所有GPU的随机种子
    np.random.seed(seed)  # 设置numpy的随机种子
    random.seed(seed)  # 设置Python的随机种子
    torch.backends.cudnn.deterministic = True  # 确保cudnn的确定性
    torch.backends.cudnn.benchmark = False  # 确保cudnn的确定性

set_seed(1111)

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
2024-08-04 12:53:29.035454: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-04 12:53:29.753398: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/home/fangmiaoNLP/.conda/envs/LZZ/lib/python3.8/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Sk

# Dataset

In [3]:
tasks, datasets, transformers = deepchem.molnet.load_clintox(splitter='scaffold', reload=True,
                                                             data_dir='./data/clintox_data',
                                                             save_dir='./data/clintox_datasets')

train_dataset, valid_dataset, test_dataset = datasets

batch_size = 1
total_batches = len(test_dataset) // batch_size

In [19]:
a = train_dataset.to_dataframe()
b = valid_dataset.to_dataframe()
c = pd.concat([a, b], ignore_index=True)
c

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X1020,X1021,X1022,X1023,X1024,y1,y2,w1,w2,ids
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,14.987342,12.463158,B([C@H](CC(C)C)NC(=O)CNC(=O)C1=C(C=CC(=C1)Cl)C...
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.071493,1.087236,C(CO)N(c1c(c(c(c(c1I)C(=O)NCC(CO)O)I)C(=O)NCC(...
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.071493,1.087236,c1c(c(cc(c1Cl)Cl)Cl)OCC#CI
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.071493,1.087236,c1c(cc(c(c1NC(=O)C(=O)[O-])Cl)NC(=O)C(=O)[O-])C#N
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.071493,1.087236,c1c(cc(c(c1S(=O)(=O)N)Cl)Cl)S(=O)(=O)N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1327,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.071493,1.087236,C[C@@H](C1=CCC23C1(C[C@H](C45C2=CC[C@H]6[C@@]4...
1328,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.071493,1.087236,C[C@@H](c1c(cncn1)F)[C@](Cn2cncn2)(c3ccc(cc3F)F)O
1329,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.071493,1.087236,C[C@@H](C(=O)N[C@H](CC(C)C)C(=O)N[C@@H](C)C(=O...
1330,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.071493,1.087236,C[C@@H]([C@@H]([C@H]1CNc2c(c(=O)nc([nH]2)N)N1)O)O


In [47]:
d = DiskDataset.from_dataframe(df=c)
d

2

In [22]:
td = train_dataset.to_dataframe()
td

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X1020,X1021,X1022,X1023,X1024,y1,y2,w1,w2,ids
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,14.987342,12.463158,B([C@H](CC(C)C)NC(=O)CNC(=O)C1=C(C=CC(=C1)Cl)C...
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.071493,1.087236,C(CO)N(c1c(c(c(c(c1I)C(=O)NCC(CO)O)I)C(=O)NCC(...
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.071493,1.087236,c1c(c(cc(c1Cl)Cl)Cl)OCC#CI
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.071493,1.087236,c1c(cc(c(c1NC(=O)C(=O)[O-])Cl)NC(=O)C(=O)[O-])C#N
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.071493,1.087236,c1c(cc(c(c1S(=O)(=O)N)Cl)Cl)S(=O)(=O)N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1179,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.071493,1.087236,CC(C[NH2+]C1CCCCC1)OC(=O)c2ccccc2
1180,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.071493,1.087236,CC(c1ccc(cc1)C(=O)c2cccs2)C(=O)[O-]
1181,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.071493,1.087236,CC(c1cc2ccccc2s1)N(C(=O)N)O
1182,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.071493,1.087236,CC(C1C(=O)NC(CSSCC(C(=O)NC(C(=O)NC(C(=O)NC(C(=...


In [39]:
a = [td.iloc[0], td.iloc[1]]
a

[X1                                                   0.0
 X2                                                   1.0
 X3                                                   0.0
 X4                                                   0.0
 X5                                                   0.0
                              ...                        
 y1                                                   0.0
 y2                                                   1.0
 w1                                             14.987342
 w2                                             12.463158
 ids    B([C@H](CC(C)C)NC(=O)CNC(=O)C1=C(C=CC(=C1)Cl)C...
 Name: 0, Length: 1029, dtype: object,
 X1                                                   0.0
 X2                                                   1.0
 X3                                                   0.0
 X4                                                   0.0
 X5                                                   0.0
                              ...

In [25]:
td['ids'].tolist()[0]

'B([C@H](CC(C)C)NC(=O)CNC(=O)C1=C(C=CC(=C1)Cl)Cl)(O)O'

# LLMs

In [4]:
class Model:
    def __init__(self, model_name_or_id: Literal["AI4Chem/ChemLLM-20B-Chat-SFT", "AI4Chem/ChemLLM-20B-Chat-DPO", "X-LANCE/ChemDFM-13B-v1.0"], **kwargs):
        assert model_name_or_id in ["AI4Chem/ChemLLM-20B-Chat-SFT", "AI4Chem/ChemLLM-20B-Chat-DPO", "X-LANCE/ChemDFM-13B-v1.0"], \
            "model must be one of 'AI4Chem/ChemLLM-20B-Chat-SFT', 'AI4Chem/ChemLLM-20B-Chat-DPO', 'X-LANCE/ChemDFM-13B-v1.0'"
        self.model_name_or_id = model_name_or_id

        self.yes_token_ids = [
            [7560,], # 7560为ChemLLM词表中Yes对应的id
            [3869,], # 3869为ChemDFM词表中Yes对应的id
            ]
        self.no_token_ids = [
            [2458, 2783],  # 2458为ChemLLM词表中No对应的id, 2783-Not
            [1939,],  # 1939为ChemDFM词表中No对应的id
            ]
        
        if "AI4Chem" in model_name_or_id:
            self.model = AutoModelForCausalLM.from_pretrained(model_name_or_id, torch_dtype=torch.float16, trust_remote_code=True, device_map="auto")
            self.tokenizer = AutoTokenizer.from_pretrained(model_name_or_id, trust_remote_code=True)
            self.yes_token_id = self.yes_token_ids[0]
            self.no_token_id = self.no_token_ids[0]

        else: # ChemDFM        
            self.model = LlamaForCausalLM.from_pretrained(model_name_or_id, torch_dtype=torch.float16, device_map="auto")
            self.tokenizer = LlamaTokenizer.from_pretrained(model_name_or_id)
            self.yes_token_id = self.yes_token_ids[1]
            self.no_token_id = self.no_token_ids[1]

        self.generation_config = GenerationConfig(
            do_sample=True,
            top_k=1,
            **kwargs,
            repetition_penalty=1.5,
            pad_token_id=self.tokenizer.eos_token_id,
            output_scores=True,
            return_dict_in_generate=True
        )

    def __call__(self, prompt: str, debug_mode: bool = False):
        inputs = self.tokenizer(prompt, return_tensors="pt").to("cuda")

        input_ids = inputs['input_ids']
        outputs = self.model.generate(**inputs, generation_config=self.generation_config)
        logits = outputs.scores
        generated_ids = outputs.sequences
        probs = [torch.softmax(log, dim=-1) for log in logits]

        output_token_ids = generated_ids[0][ len(input_ids[0]): ]
        response = self.tokenizer.decode(output_token_ids, skip_special_tokens=True)
        
        if debug_mode:
            for i, token_id in enumerate(output_token_ids):
                token_prob = probs[i][0, token_id].item()
                print(f"Token ID: {token_id}, Probability: {token_prob}")

        total_yesp, total_nop = 0., 0.
        for x in self.yes_token_id:
            total_yesp += probs[0][0, x].item()
        for y in self.no_token_id:
            total_nop += probs[0][0, y].item()
        
        sump = (total_yesp + total_nop) + 1e-14
        y_score = total_yesp / sump

        return [response], [y_score]

# Prompt

In [6]:
class BasePrompter(object):
    def __init__(self, system_instruction: str = "", template: str = "", use_canonicalize_smiles: bool = False, verbose: bool = False):
        self.system_instruction = system_instruction
        self.template = template
        self.use_canonicalize_smiles = use_canonicalize_smiles
        self.verbose = verbose

    def generate_prompt(self, query_smiles):
        raise NotImplementedError
    
    def canonicalize_smiles(self, smiles):
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return None
        return Chem.MolToSmiles(mol, isomericSmiles=True, kekuleSmiles=False)
    
class ZeroShotPrompter(BasePrompter):
    def __init__(self, system_instruction: str = "", template: str = "", use_canonicalize_smiles: bool = False, verbose: bool = False):
        super().__init__(system_instruction, template, use_canonicalize_smiles, verbose)

    @staticmethod
    def generate_prompt(self, query_smiles):
        if self.use_canonicalize_smiles:
            query_smiles = self.canonicalize_smiles(query_smiles)

        prompt = f"{self.template}\nSMILES:{query_smiles}\nAnswer:"

        if self.verbose:
            print(prompt)

        return prompt

class FewShotPrompter(BasePrompter):
    def __init__(self, system_instruction: str = "", template: str = "", use_canonicalize_smiles: bool = False, verbose: bool = False, *, sample_dataset, sample_mode: Literal['random', 'cosinesimilarity'], sample_num: int):
        super(FewShotPrompter, self).__init__(system_instruction, template, use_canonicalize_smiles, verbose)
        
        self.sample_dataset = sample_dataset.to_dataframe()
        if sample_mode not in ['random', 'cosinesimilarity']:
            raise ValueError("mode must be either 'random' or 'cosinesimilarity'")
        self.sample_mode = sample_mode
        self.sample_num = sample_num

    def get_demonstrations(self):
        if self.sample_mode == 'random':
            return self.random_sample_examples()

    def random_sample_examples(self):
        # y2 = CT_TOX
        positive_examples = self.sample_dataset[self.sample_dataset["y2"] == 1].sample(int(self.sample_num/2))
        negative_examples = self.sample_dataset[self.sample_dataset["y2"] == 0].sample(int(self.sample_num/2))

        smiles = positive_examples["ids"].tolist() + negative_examples["ids"].tolist()
        smiles = [self.canonicalize_smiles(i) for i in smiles]
        class_label = positive_examples["y2"].tolist() + negative_examples["y2"].tolist()
        #convert 1 to "Yes" and 0 to "No"" in class_label
        class_label = ["Yes" if i == 1 else "No" for i in class_label]
        bace_examples = list(zip(smiles, class_label))
        return bace_examples

    def generate_prompt(self, query_smiles):
        if self.use_canonicalize_smiles:
            query_smiles = self.canonicalize_smiles(query_smiles)

        few_shot = ""
        demonstrations = self.get_demonstrations()
        for example in demonstrations:
            few_shot += f"SMILES:{example[0]}\nClinically-trail-toxic:{example[-1]}\n"

        prompt = f"{self.template}\n{few_shot}SMILES:{query_smiles}\nClinically-trail-toxic:"

        if self.verbose:
            print(prompt)
            
        return prompt
    
class FewShotPrompter1(FewShotPrompter):
    def generate_prompt(self, query_smiles):
        if self.use_canonicalize_smiles:
            query_smiles = self.canonicalize_smiles(query_smiles)

        few_shot = ""
        demonstrations = self.get_demonstrations()
        for example in demonstrations:
            few_shot += f"SMILES:{example[0]}\nAnswer:{example[-1]}\n"

        prompt = f"{self.template}\n{few_shot}Is this molecule Clinically-trail-Toxic (Yes) or Not Clinically-trail-toxic (No)?\nSMILES:{query_smiles}\nAnswer:"

        if self.verbose:
            print(prompt)
            
        return prompt

# Inference

In [7]:
def main(dataset: Any, 
         batch_size: int, 
         total_batches: int, 
         model: Callable[[str, bool], Tuple],
         prompt_generator: Callable[[str], str],
         ):
    y_trues = []
    y_scores = []
    responses = []
    cnt = 0
    
    for X, Y, W, ids in tqdm(dataset.iterbatches(batch_size=batch_size), total=total_batches):
        input_X = [prompt_generator(id) for id in ids]
        
        y_trues.extend(Y[:, -1])

        bs_responses, bs_y_scores = model(input_X)
        
        print(bs_responses, bs_y_scores)

        responses.extend(bs_responses)
        y_scores.extend(bs_y_scores)

        cnt += 1

        # if cnt > 3:
        #     break

    print(responses)
    print(y_trues)
    print(y_scores)
    print(cnt)

    roc = roc_auc_score(y_trues, y_scores)
    print(roc)

In [8]:
zero_shot_prompt = \
    "You are an expert chemist, your task is to predict the property of molecule using your experienced chemical property prediction knowledge.\nPlease strictly follow the format, no other information can be provided. Given the SMILES string of a molecule, the task focuses on predicting molecular properties, specifically wether a molecule is Clinically-trail-Toxic(Yes) or Not Clinically-trail-toxic (No) based on the SMILES string representation of each molecule. The task is to predict the binary label for a given molecule, please answer with only 'Yes' or 'No'."


In [9]:
few_shot_prompt = \
    "Please strictly follow the format, no other information can be provided. Given the SMILES string of a molecule, the task focuses on predicting molecular properties, specifically wether a molecule is Clinically-trail-Toxic (Yes) or Not Clinically-trail-toxic (No). The task is to predict the binary label for a given molecule, please answer with only 'Yes' or 'No'."

In [6]:
model = Model(model_name_or_id='AI4Chem/ChemLLM-20B-Chat-SFT',
              temperature=0.9,
              max_new_tokens=10)

Loading checkpoint shards:   0%|          | 0/41 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
model = Model(model_name_or_id='X-LANCE/ChemDFM-13B-v1.0',
              temperature=0.9,
              max_new_tokens=10)

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Some weights of LlamaForCausalLM were not initialized from the model checkpoint at X-LANCE/ChemDFM-13B-v1.0 and are newly initialized: ['model.layers.33.self_attn.rotary_emb.inv_freq', 'model.layers.31.self_attn.rotary_emb.inv_freq', 'model.layers.11.self_attn.rotary_emb.inv_freq', 'model.layers.6.self_attn.rotary_emb.inv_freq', 'model.layers.10.self_attn.rotary_emb.inv_freq', 'model.layers.29.self_attn.rotary_emb.inv_freq', 'model.layers.20.self_attn.rotary_emb.inv_freq', 'model.layers.22.self_attn.rotary_emb.inv_freq', 'model.layers.30.self_attn.rotary_emb.inv_freq', 'model.layers.27.self_attn.rotary_emb.inv_freq', 'model.layers.5.self_attn.rotary_emb.inv_freq', 'model.layers.28.self_attn.rotary_emb.inv_freq', 'model.layers.1.self_attn.rotary_emb.inv_freq', 'model.layers.4.self_attn.rotary_emb.inv_freq', 'model.layers.21.self_attn.rotary_emb.inv_freq', 'model.layers.23.self_attn.rotary_emb.inv_freq', 'model.layers.0.self_attn.rotary_emb.inv_freq', 'model.layers.15.self_attn.rotary_em

# Verbose & Test

In [13]:
zero_shot_prompter = ZeroShotPrompter(template=zero_shot_prompt, use_canonicalize_smiles=False, verbose=True)
a = zero_shot_prompter.generate_prompt('O=C(OCc1ccccc1)c1ccccc1')

You are an expert chemist, your task is to predict the property of molecule using your experienced chemical property prediction knowledge.
Please strictly follow the format, no other information can be provided. Given the SMILES string of a molecule, the task focuses on predicting molecular properties, specifically wether a molecule is Clinically-trail-Toxic(Yes) or Not Clinically-trail-toxic (No) based on the SMILES string representation of each molecule. The task is to predict the binary label for a given molecule, please answer with only 'Yes' or 'No'.
SMILES:O=C(OCc1ccccc1)c1ccccc1
Clinically-trail-toxic:


In [133]:
few_shot_prompter = FewShotPrompter(template=few_shot_prompt, use_canonicalize_smiles=True, verbose=True,
                                    sample_dataset=train_dataset, sample_mode='random', sample_num=2)

b = few_shot_prompter.generate_prompt('NS(=O)(=O)Cc1noc2ccccc12')

You are an expert chemist, your task is to predict the property of molecule using your experienced chemical property prediction knowledge.
Given the SMILES string of a molecule, the task focuses on predicting molecular properties, specifically wether a molecule is Clinically-trail-Toxic(Yes) or Not Clinically-trail-toxic (No) based on the SMILES string representation of each molecule. The task is to predict the binary label for a given molecule, please answer with only 'Yes' or 'No'. Please strictly follow the format, no other information can be provided. A few templates are provided in the beginning.
SMILES:O=C(CCCCCCC(=O)Nc1ccccc1)NO
Clinically-trail-toxic:Yes
SMILES:CC(C)([NH3+])Cc1ccccc1
Clinically-trail-toxic:No
SMILES:NS(=O)(=O)Cc1noc2ccccc12
Clinically-trail-toxic:


In [ ]:
test = \
    "You are an expert chemist, your task is to predict the property of molecule using your experienced chemical property prediction knowledge.\nGiven the SMILES string of a molecule, the task focuses on predicting molecular properties, specifically wether a molecule is Clinically-trail-Toxic(Yes) or Not Clinically-trail-toxic (No) based on the SMILES string representation of each molecule.The task is to predict the binary label for a given molecule, please answer with only 'Yes' or 'No', 'Unknown' is not allowed.Please strictly follow the format, no other information can be provided.\nSMILES: O=C(OCc1ccccc1)c1ccccc1\nClinically-trail-toxic: "

model(test)

In [95]:
test = \
    "You are an expert chemist, your task is to predict the property of molecule using your experienced chemical property prediction knowledge.\n\
    Given the SMILES string of a molecule, the task focuses on predicting molecular properties, specifically wether a molecule is Clinically-trail-Toxic(Yes) or Not Clinically-trail-toxic (No) based on the SMILES string representation of each molecule. \
    The task is to predict the binary label for a given molecule, please answer with only 'Yes' or 'No', 'Unknown' is not allowed. \
    Please strictly follow the format, no other information can be provided.\n\
    SMILES: O=C(OCc1ccccc1)c1ccccc1\n\
    Clinically-trail-toxic:"

model(test)

You are an expert chemist, your task is to predict the property of molecule using your experienced chemical property prediction knowledge.
    Given the SMILES string of a molecule, the task focuses on predicting molecular properties, specifically wether a molecule is Clinically-trail-Toxic(Yes) or Not Clinically-trail-toxic (No) based on the SMILES string representation of each molecule.     The task is to predict the binary label for a given molecule, please answer with only 'Yes' or 'No', 'Unknown' is not allowed.     Please strictly follow the format, no other information can be provided.
    SMILES: O=C(OCc1ccccc1)c1ccccc1
    Clinically-trail-toxic: Yes


([' Yes'], [0.0])

# Random

## 1. Few-shot with 2 random samples

In [36]:
few_shot_prompter = FewShotPrompter(template=few_shot_prompt, use_canonicalize_smiles=True,
                                    sample_dataset=train_dataset, sample_mode='random', sample_num=2)

main(dataset=test_dataset,
     batch_size=batch_size,
     total_batches=total_batches,
     model=model,
     prompt_generator=few_shot_prompter.generate_prompt)

  1%|          | 1/148 [00:10<26:02, 10.63s/it]

['Not found'] [0.0]


  1%|▏         | 2/148 [00:21<26:47, 11.01s/it]

['Not Clinical trial toxic'] [0.0]


  2%|▏         | 3/148 [00:32<25:51, 10.70s/it]

...

['Not Clinical trial toxic'] [0.0]


 99%|█████████▉| 147/148 [26:08<00:11, 11.70s/it]

[' No'] [0.0]


100%|██████████| 148/148 [26:18<00:00, 10.67s/it]

['Not found'] [0.0]
['Not found', 'Not Clinical trial toxic', ' No', 'Not found', 'Not found', 'Not Clinical trial toxic', 'Not found', 'Not Clinical trial toxic', 'Not clinistically toxic', 'Not found', 'Not found', ' No', 'Not Now', ' No', 'Not found', ' No', 'Not clinistically trail toxic', 'Not Clinical trial toxic', ' No', 'Not found', ' No', 'Not Clinical trial toxic', 'Not found', ' No', 'Not Clinical trial toxic', ' No', 'Not Clinical trial toxic', 'Not found', ' No', 'Not found', 'Not found', 'Not found', 'Not Clinical trial toxic', ' No', 'Not Clinical trial toxic', 'Not found', 'Not found', ' No', 'Not found', ' No', 'Not in my opinion', 'Not found', ' No', 'Not found', ' No', 'Not Clinical trial toxic', 'Not found', ' No', 'Not clinistically toxic', ' No', 'Not found', 'Not found', ' No', ' No', 'Not Clinical trial toxic', 'Not Clinical trial toxic', 'Not Clinical trial toxic', ' No', 'Not found', 'Not clinistically trail toxic', ' No', 'Not found', 'Not found', 'Not found'

## 2. Few-shot with 4 random samples

In [37]:
few_shot_prompter = FewShotPrompter(template=few_shot_prompt, use_canonicalize_smiles=True,
                                    sample_dataset=train_dataset, sample_mode='random', sample_num=4)

main(dataset=test_dataset,
     batch_size=batch_size,
     total_batches=total_batches,
     model=model,
     prompt_generator=few_shot_prompter.generate_prompt)

  1%|          | 1/148 [00:12<30:31, 12.46s/it]

[' No'] [0.0]


  1%|▏         | 2/148 [00:27<34:21, 14.12s/it]

['Not Clinical trial toxic'] [0.0]


  2%|▏         | 3/148 [00:40<32:11, 13.32s/it]

...

['Not clinistically toxic'] [0.0]


 99%|█████████▉| 147/148 [34:42<00:14, 14.61s/it]

['Not from my perspective, the answer is No'] [0.0]


100%|██████████| 148/148 [34:59<00:00, 14.19s/it]

['Not clinistically toxic'] [0.0]
[' No', 'Not Clinical trial toxic', 'Not found', ' No', ' No', ' No', 'Not clinistically trail toxic', ' No', 'Not found', ' No', ' No', ' No', 'Not found', 'Not clinistically toxic', ' No', 'Not clinistically toxic', 'Not from my perspective, theanswer would appear as', ' No', ' No', ' No', ' No', 'Not clinistically toxic', ' No', ' No', 'Not clinistically trail toxic', 'Not clinistically trail toxic', ' No', ' No', ' No', 'Not clinistically trail toxic', 'Not Clinical trial toxic', ' No', 'Not from my perspective,the answering could have been', 'Not found', 'Not found', 'Not in my opinion', 'Not clinistically toxic', 'Not in my opinion', 'Not Clinical trial toxic', 'Not Clinical trial toxic', ' No', ' No', ' No', 'Not from my perspective, the answer is No', 'Not Clinical trial toxic', ' No', ' No', 'Not clinistically trail toxic', ' No', 'Not Clinical trial toxic', ' No', ' No', 'Not clinistically trail toxic', 'Not clinistically toxic', ' No', ' No'

## 3. Few-shot with 8 random samples

In [38]:
few_shot_prompter = FewShotPrompter(template=few_shot_prompt, use_canonicalize_smiles=True,
                                    sample_dataset=train_dataset, sample_mode='random', sample_num=8)

main(dataset=test_dataset,
     batch_size=batch_size,
     total_batches=total_batches,
     model=model,
     prompt_generator=few_shot_prompter.generate_prompt)

  1%|          | 1/148 [00:18<45:15, 18.47s/it]

[' No'] [0.0]


  1%|▏         | 2/148 [00:38<47:05, 19.35s/it]

['Not clinistically trail toxic'] [0.0]


  2%|▏         | 3/148 [00:58<47:37, 19.71s/it]

...

[' No'] [0.0]


 99%|█████████▉| 147/148 [51:47<00:20, 20.83s/it]

[' No'] [0.0]


100%|██████████| 148/148 [52:10<00:00, 21.15s/it]

[' No'] [0.0]
[' No', 'Not clinistically trail toxic', ' No', ' No', ' No', 'Not clinistically trail toxic', 'Not clinistically trail toxic', ' No', ' No', 'Not clinistically trail toxic', ' No', ' No', ' No', ' No', ' No', ' No', ' No', 'Not clin', 'Not clinistically toxic', 'Not clinistically toxic', ' No', 'Not clinistically trail toxic', ' No', ' No', ' No', 'Not clinistically trail toxic', 'Not clinistically trail toxic', 'Not clinistically toxic', ' No', 'Not clinistically toxic', 'Not clinistically trail toxic', 'Not clinistically trail toxic', ' No', ' No', ' No', ' No', ' No', ' No', ' No', ' No', ' No', ' No', ' No', ' No', ' No', ' No', ' No', ' No', ' No', 'Not clinistically toxic', ' No', ' No', 'Not clinistically trail toxic', ' No', ' No', 'Not clinistically toxic', 'Not clinistically trail toxic', 'Not clinistically trail toxic', 'Not clinistically trail toxic', 'Not from my perspective,the answer would appear as', 'Not clinistically trail toxic', ' No', 'Not clinistica

## 3.5 Zero-shot

In [7]:
zero_shot_prompt = \
    "You are an expert chemist, your task is to predict the property of molecule using your experienced chemical property prediction knowledge.\nPlease strictly follow the format, no other information can be provided. Given the SMILES string of a molecule, the task focuses on predicting molecular properties, specifically wether a molecule is Clinically-trail-Toxic(Yes) or Not Clinically-trail-toxic (No) based on the SMILES string representation of each molecule. The task is to predict the binary label for a given molecule, please answer with only 'Yes' or 'No'."

class ZeroShotPrompter1(BasePrompter):
    def generate_prompt(self, query_smiles):
        if self.use_canonicalize_smiles:
            query_smiles = self.canonicalize_smiles(query_smiles)

        prompt = f"{self.template}\nSMILES:{query_smiles}\nAnswer:"

        if self.verbose:
            print(prompt)

        return prompt

zero_shot_prompter = ZeroShotPrompter1(template=zero_shot_prompt, use_canonicalize_smiles=True)

main(dataset=test_dataset,
     batch_size=batch_size,
     total_batches=total_batches,
     model=model,
     prompt_generator=zero_shot_prompter.generate_prompt)

  1%|          | 1/148 [00:06<17:04,  6.97s/it]

[' No'] [0.0]


  1%|▏         | 2/148 [00:13<16:29,  6.78s/it]

[' No'] [0.0]


  2%|▏         | 3/148 [00:19<15:50,  6.55s/it]

...

[' No'] [0.0]


 99%|█████████▉| 147/148 [15:47<00:06,  6.37s/it]

[' No'] [0.0]


100%|██████████| 148/148 [15:53<00:00,  6.44s/it]

[' No'] [0.0]
[' No', ' No', ' No', ' No', ' No', ' No', ' No', ' Yes', ' No', ' No', ' No', ' No', ' Yes', ' No', ' No', ' Yes', ' Yes', ' No', ' No', ' No', ' No', ' Yes', ' No', ' No', ' No', ' No', ' No', ' No', ' Yes', ' No', ' Yes', ' Yes', ' Yes', ' No', ' Yes', ' No', ' No', ' Yes', ' No', ' Yes', ' No', ' No', ' No', ' No', ' Yes', ' No', ' Yes', ' No', ' Yes', ' No', ' Yes', ' No', ' Yes', ' No', ' No', ' Yes', ' No', ' Yes', ' No', ' No', ' No', ' No', ' No', ' Yes', ' No', ' Yes', ' No', ' No', ' No', ' No', ' No', ' No', ' No', ' No', ' No', ' Yes', ' No', ' No', ' No', ' No', ' Yes', ' No', ' Yes', ' No', ' No', ' No', ' No', ' No', ' Yes', ' No', ' No', ' Yes', ' No', ' Yes', ' No', ' No', ' Yes', ' No', ' Yes', ' No', ' No', ' No', ' No', ' No', ' No', ' Yes', ' Yes', ' Yes', ' Yes', ' No', ' No', ' Yes', ' No', ' No', ' No', ' Yes', ' No', ' No', ' No', ' No', ' No', ' No', ' Yes', ' No', ' No', ' No', ' No', ' No', ' No', ' No', ' No', ' No', ' No', ' No', ' Yes', ' Y

## 4. Few-shot with 2 random exapmles

In [54]:
few_shot_prompt = \
    "Please strictly follow the format, no other information can be provided. Given the SMILES string of a molecule, the task focuses on predicting molecular properties, specifically wether a molecule is Clinically-trail-Toxic (Yes) or Not Clinically-trail-toxic (No). The task is to predict the binary label for a given molecule, please answer with only 'Yes' or 'No'."

class FewShotPrompter1(FewShotPrompter):
    def generate_prompt(self, query_smiles):
        if self.use_canonicalize_smiles:
            query_smiles = self.canonicalize_smiles(query_smiles)

        few_shot = ""
        demonstrations = self.get_demonstrations()
        for example in demonstrations:
            few_shot += f"SMILES:{example[0]}\nAnswer:{example[-1]}\n"

        prompt = f"{self.template}\n{few_shot}Is this molecule Clinically-trail-Toxic (Yes) or Not Clinically-trail-toxic (No)?\nSMILES:{query_smiles}\nAnswer:"

        if self.verbose:
            print(prompt)
            
        return prompt

few_shot_prompter = FewShotPrompter1(template=few_shot_prompt, use_canonicalize_smiles=True,
                                    sample_dataset=train_dataset, sample_mode='random', sample_num=2)

main(dataset=test_dataset,
     batch_size=batch_size,
     total_batches=total_batches,
     model=model,
     prompt_generator=few_shot_prompter.generate_prompt)

  1%|          | 1/148 [00:08<20:28,  8.35s/it]

[' No'] [0.0]


  1%|▏         | 2/148 [00:18<22:45,  9.35s/it]

[' No'] [0.0]


  2%|▏         | 3/148 [00:26<21:17,  8.81s/it]

...

[' Yes'] [0.99999999999999]


 99%|█████████▉| 147/148 [21:54<00:09,  9.94s/it]

[' No'] [0.0]


100%|██████████| 148/148 [22:02<00:00,  8.94s/it]

[' Yes'] [0.99999999999999]
[' No', ' No', ' No', ' No', ' No', ' No', ' No', ' No', ' Yes', ' Yes', ' No', ' No', ' Yes', ' No', ' No', ' No', ' Yes', ' Yes', ' Yes', ' No', ' Yes', ' No', ' Yes', ' Yes', ' Yes', ' No', ' Yes', ' No', ' No', ' Yes', ' No', ' No', ' No', ' Yes', ' No', ' No', ' No', ' No', ' No', ' No', ' Yes', ' Yes', ' Yes', ' No', ' Yes', ' No', ' No', ' No', ' Yes', ' No', ' No', ' No', ' Yes', ' No', ' No', ' No', ' No', ' Yes', ' No', ' No', ' No', ' No', ' No', ' Yes', ' No', ' Yes', ' Yes', ' No', ' Yes', ' No', ' No', ' No', ' No', ' Yes', ' No', ' No', ' No', ' No', ' No', ' No', ' Yes', ' Yes', ' No', ' No', ' No', ' No', ' No', ' No', ' No', ' No', ' Yes', ' Yes', ' No', ' No', ' No', ' Yes', ' Yes', ' No', ' No', ' Yes', ' No', ' No', ' Yes', ' Yes', ' Yes', ' No', ' No', ' Yes', ' No', ' Yes', ' Yes', ' No', ' Yes', ' No', ' No', ' No', ' No', ' Yes', ' No', ' No', ' Yes', ' No', ' No', ' No', ' No', ' Yes', ' No', ' Yes', ' No', ' No', ' No', ' No', ' No

## 5. Few-shot with 4 random exapmles

In [55]:
few_shot_prompter = FewShotPrompter1(template=few_shot_prompt, use_canonicalize_smiles=True,
                                    sample_dataset=train_dataset, sample_mode='random', sample_num=4)

main(dataset=test_dataset,
     batch_size=batch_size,
     total_batches=total_batches,
     model=model,
     prompt_generator=few_shot_prompter.generate_prompt)

  1%|          | 1/148 [00:10<25:10, 10.28s/it]

[' No'] [0.0]


  1%|▏         | 2/148 [00:18<22:13,  9.14s/it]

[' Yes'] [0.99999999999999]


  2%|▏         | 3/148 [00:28<23:22,  9.67s/it]

...

[' No'] [0.0]


 99%|█████████▉| 147/148 [29:23<00:11, 11.71s/it]

[' No'] [0.0]


100%|██████████| 148/148 [29:33<00:00, 11.99s/it]

[' Yes'] [0.99999999999999]
[' No', ' Yes', ' No', ' No', ' No', ' Yes', ' Yes', ' No', ' Yes', ' No', ' No', ' Yes', ' No', ' No', ' No', ' Yes', ' No', ' No', ' Yes', ' Yes', ' Yes', ' Yes', ' Yes', ' Yes', ' Yes', ' No', ' Yes', ' Yes', ' Yes', ' Yes', ' Yes', ' No', ' No', ' No', ' Yes', ' No', ' No', ' No', ' No', ' Yes', ' No', ' Yes', ' No', ' No', ' Yes', ' No', ' No', ' No', ' No', ' No', ' No', ' No', ' No', ' Yes', ' Yes', ' No', ' Yes', ' No', ' No', ' Yes', ' Yes', ' Yes', ' Yes', ' Yes', ' No', ' Yes', ' Yes', ' Yes', ' No', ' Yes', ' Yes', ' Yes', ' No', ' No', ' No', ' No', ' No', ' No', ' No', ' Yes', ' No', ' No', ' No', ' No', ' No', ' No', ' Yes', ' No', ' Yes', ' No', ' Yes', ' No', ' No', ' No', ' Yes', ' No', ' No', ' No', ' No', ' Yes', ' No', ' No', ' No', ' No', ' No', ' No', ' Yes', ' No', ' No', ' No', ' No', ' No', ' Yes', ' No', ' Yes', ' Yes', ' No', ' No', ' No', ' Yes', ' No', ' No', ' Yes', ' No', ' No', ' No', ' No', ' No', ' No', ' Yes', ' No', ' No'

## 6. Few-shot with 8 random exapmles

In [56]:
few_shot_prompter = FewShotPrompter1(template=few_shot_prompt, use_canonicalize_smiles=True,
                                    sample_dataset=train_dataset, sample_mode='random', sample_num=8)

main(dataset=test_dataset,
     batch_size=batch_size,
     total_batches=total_batches,
     model=model,
     prompt_generator=few_shot_prompter.generate_prompt)

  1%|          | 1/148 [00:22<54:46, 22.36s/it]

[' No'] [0.0]


  1%|▏         | 2/148 [00:38<45:35, 18.74s/it]

[' Yes'] [0.99999999999999]


  2%|▏         | 3/148 [00:58<46:49, 19.38s/it]

...

[' No'] [0.0]


 99%|█████████▉| 147/148 [44:47<00:17, 17.02s/it]

[' No'] [0.0]


100%|██████████| 148/148 [45:04<00:00, 18.27s/it]

[' No'] [0.0]
[' No', ' Yes', ' No', ' No', ' Yes', ' Yes', ' No', ' No', ' Yes', ' No', ' No', ' No', ' No', ' Yes', ' No', ' Yes', ' No', ' Yes', ' Yes', ' Yes', ' No', ' No', ' No', ' Yes', ' No', ' Yes', ' No', ' Yes', ' No', ' No', ' Yes', ' No', ' Yes', ' No', ' Yes', ' No', ' No', ' Yes', ' No', ' No', ' No', ' Yes', ' Yes', ' No', ' Yes', ' No', ' No', ' No', ' No', ' Yes', ' No', ' No', ' No', ' No', ' No', ' No', ' No', ' Yes', ' Yes', ' Yes', ' Yes', ' Yes', ' Yes', ' No', ' No', ' No', ' No', ' Yes', ' No', ' Yes', ' No', ' Yes', ' No', ' Yes', ' No', ' No', ' No', ' No', ' Yes', ' No', ' Yes', ' No', ' No', ' No', ' No', ' No', ' Yes', ' Yes', ' Yes', ' Yes', ' No', ' Yes', ' No', ' Yes', ' No', ' Yes', ' No', ' No', ' No', ' No', ' No', ' No', ' No', ' Yes', ' Yes', ' Yes', ' No', ' Yes', ' Yes', ' No', ' Yes', ' Yes', ' Yes', ' No', ' Yes', ' No', ' No', ' No', ' No', ' No', ' No', ' Yes', ' No', ' No', ' No', ' Yes', ' No', ' No', ' No', ' No', ' No', ' No', ' Yes', ' N

## 7. Few-shot with 8 random examples different temperature（0.8）

In [13]:
few_shot_prompt = \
    "Please strictly follow the format, no other information can be provided. Given the SMILES string of a molecule, the task focuses on predicting molecular properties, specifically wether a molecule is Clinically-trail-Toxic (Yes) or Not Clinically-trail-toxic (No). The task is to predict the binary label for a given molecule, please answer with only 'Yes' or 'No'."

class FewShotPrompter1(FewShotPrompter):
    def generate_prompt(self, query_smiles):
        if self.use_canonicalize_smiles:
            query_smiles = self.canonicalize_smiles(query_smiles)

        few_shot = ""
        demonstrations = self.get_demonstrations()
        for example in demonstrations:
            few_shot += f"SMILES:{example[0]}\nAnswer:{example[-1]}\n"

        prompt = f"{self.template}\n{few_shot}Is this molecule Clinically-trail-Toxic (Yes) or Not Clinically-trail-toxic (No)?\nSMILES:{query_smiles}\nAnswer:"

        if self.verbose:
            print(prompt)
            
        return prompt

few_shot_prompter = FewShotPrompter1(template=few_shot_prompt, use_canonicalize_smiles=True,
                                    sample_dataset=train_dataset, sample_mode='random', sample_num=8)

main(dataset=test_dataset,
     batch_size=batch_size,
     total_batches=total_batches,
     model=model,
     prompt_generator=few_shot_prompter.generate_prompt)

  1%|          | 1/148 [00:16<40:17, 16.44s/it]

[' Yes'] [0.99999999999999]


  1%|▏         | 2/148 [00:32<39:41, 16.31s/it]

[' Yes'] [0.99999999999999]


  2%|▏         | 3/148 [00:48<39:04, 16.17s/it]

...

[' No'] [0.0]


 99%|█████████▉| 147/148 [44:31<00:15, 15.61s/it]

[' No'] [0.0]


100%|██████████| 148/148 [44:49<00:00, 18.17s/it]

[' Yes'] [0.99999999999999]
[' Yes', ' Yes', ' No', ' No', ' No', ' Yes', ' No', ' Yes', ' No', ' No', ' No', ' No', ' No', ' No', ' Yes', ' Yes', ' No', ' No', ' No', ' No', ' No', ' Yes', ' No', ' No', ' No', ' No', ' Yes', ' Yes', ' No', ' Yes', ' No', ' No', ' Yes', ' Yes', ' No', ' No', ' Yes', ' Yes', ' No', ' Yes', ' No', ' No', ' No', ' Yes', ' Yes', ' Yes', ' No', ' No', ' No', ' Yes', ' No', ' No', ' Yes', ' No', ' No', ' Yes', ' Yes', ' Yes', ' No', ' Yes', ' No', ' Yes', ' No', ' No', ' No', ' No', ' Yes', ' Yes', ' No', ' No', ' No', ' No', ' No', ' No', ' No', ' No', ' Yes', ' Yes', ' No', ' Yes', ' Yes', ' No', ' No', ' No', ' No', ' No', ' Yes', ' Yes', ' No', ' Yes', ' No', ' No', ' No', ' No', ' No', ' Yes', ' No', ' Yes', ' No', ' No', ' No', ' No', ' No', ' Yes', ' No', ' No', ' Yes', ' Yes', ' No', ' No', ' No', ' Yes', ' Yes', ' No', ' No', ' No', ' Yes', ' Yes', ' No', ' No', ' Yes', ' Yes', ' Yes', ' No', ' Yes', ' Yes', ' No', ' No', ' Yes', ' No', ' No', ' No'

## 8. Few-shot with 8 random examples and a slightly different prompt

In [11]:
few_shot_prompt = \
    "Please strictly follow the format, no other information can be provided. Given the SMILES string of a molecule, the task focuses on predicting molecular properties, specifically wether a molecule is Clinically-trail-Toxic (Yes) or Not Clinically-trail-toxic (No). The task is to predict the binary label for a given molecule, please answer with only 'Yes' or 'No'."

class FewShotPrompter2(FewShotPrompter1):
    def random_sample_examples(self):
        # y2 = CT_TOX
        positive_examples = self.sample_dataset[self.sample_dataset["y2"] == 1].sample(int(self.sample_num/2))
        negative_examples = self.sample_dataset[self.sample_dataset["y2"] == 0].sample(int(self.sample_num/2))

        smiles = positive_examples["ids"].tolist() + negative_examples["ids"].tolist()
        smiles = [self.canonicalize_smiles(i) for i in smiles]
        class_label = positive_examples["y2"].tolist() + negative_examples["y2"].tolist()
        #convert 1 to "Yes" and 0 to "No"" in class_label
        class_label = ["Yes" if i == 1 else "No" for i in class_label]
        bace_examples = list(zip(smiles, class_label))
        random.shuffle(bace_examples)
        return bace_examples
    
    # def generate_prompt(self, query_smiles):
    #     if self.use_canonicalize_smiles:
    #         query_smiles = self.canonicalize_smiles(query_smiles)

    #     few_shot = ""
    #     demonstrations = self.get_demonstrations()
    #     for example in demonstrations:
    #         few_shot += f"SMILES:{example[0]}\nAnswer:{example[-1]}\n"

    #     prompt = f"{self.template}\n{few_shot}Is this molecule Clinically-trail-Toxic (Yes) or Not Clinically-trail-toxic (No)?\nSMILES:{query_smiles}\nAnswer:"

    #     if self.verbose:
    #         print(prompt)
            
    #     return prompt

few_shot_prompter = FewShotPrompter2(template=few_shot_prompt, use_canonicalize_smiles=True,
                                    sample_dataset=train_dataset, sample_mode='random', sample_num=8)

main(dataset=test_dataset,
     batch_size=batch_size,
     total_batches=total_batches,
     model=model,
     prompt_generator=few_shot_prompter.generate_prompt)
# print(few_shot_prompter.generate_prompt('NS(=O)(=O)Cc1noc2ccccc12'))

  0%|          | 0/148 [00:00<?, ?it/s]

  1%|          | 1/148 [00:20<50:55, 20.78s/it]

[' No'] [0.0]


  1%|▏         | 2/148 [00:33<38:56, 16.00s/it]

[' No'] [0.0]


...

[' No'] [0.0]


 99%|█████████▉| 147/148 [45:29<00:18, 18.14s/it]

[' Yes'] [0.499999999999995]


100%|██████████| 148/148 [45:47<00:00, 18.56s/it]

[' No'] [0.0]
[' No', ' No', ' No', ' No', ' No', ' Yes', ' No', ' No', ' No', ' No', ' No', 'Not Clinical trial toxic', ' No', ' No', ' Yes', 'Not Clinical trial toxic', ' Yes', ' No', ' No', ' No', ' No', ' No', ' No', ' No', ' No', ' Yes', ' No', 'Not Clinical trial toxic', ' No', ' No', ' No', ' No', ' No', ' Yes', ' Yes', ' No', ' Yes', ' Yes', ' No', ' Yes', ' No', ' No', ' Yes', ' No', ' Yes', ' No', ' Yes', ' No', ' Yes', ' No', ' No', ' No', ' No', ' No', ' No', 'Not Clinical trial toxic', ' No', ' Yes', ' No', ' No', ' Yes', ' Yes', ' No', ' No', 'Not Clinical trial toxic', ' Yes', ' Yes', ' No', ' No', ' No', ' No', ' No', ' No', 'Not Clinical trial toxic', ' No', ' No', ' Yes', ' No', ' Yes', ' Yes', ' Yes', ' No', ' Yes', ' No', ' No', ' No', ' No', ' Yes', ' No', ' No', ' No', ' Yes', ' No', ' Yes', ' No', ' Yes', ' No', ' No', ' No', ' No', 'Not Clinical trial toxic', ' No', 'Not Clinical trial toxic', ' No', ' No', ' Yes', ' No', 'Not Clinical trial toxic', ' Yes', ' No

## 9. ChemDFM few-shot with 8 random examples

In [12]:
few_shot_prompter = FewShotPrompter1(template=few_shot_prompt, use_canonicalize_smiles=True,
                                    sample_dataset=train_dataset, sample_mode='random', sample_num=8)

main(dataset=test_dataset,
     batch_size=batch_size,
     total_batches=total_batches,
     model=model,
     prompt_generator=few_shot_prompter.generate_prompt)

  0%|          | 0/148 [00:00<?, ?it/s]

  1%|          | 1/148 [00:16<41:04, 16.77s/it]

['No'] [0.0]


  1%|▏         | 2/148 [00:29<34:45, 14.28s/it]

['Yes'] [0.99999999999999]


...

['No'] [0.0]


 99%|█████████▉| 147/148 [35:32<00:14, 14.15s/it]

['Yes'] [0.99999999999999]


100%|██████████| 148/148 [35:47<00:00, 14.51s/it]

['No'] [0.0]
['No', 'Yes', 'No', 'No', 'No', 'Yes', 'No', 'No', 'Yes', 'Yes', 'No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'No', 'No', 'No', 'No', 'No', 'Yes', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'No', 'No', 'No', 'Yes', 'No', 'Yes', 'No', 'No', 'Yes', 'No', 'No', 'No', 'No', 'No', 'No', 'Yes', 'Yes', 'No', 'Yes', 'No', 'No', 'No', 'No', 'No', 'Yes', 'No', 'Yes', 'Yes', 'No', 'No', 'No', 'No', 'Yes', 'Yes', 'No', 'Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'No', 'Yes', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'Yes', 'No', 'Yes', 'No', 'No', 'No', 'No', 'Yes', 'No', 'No', 'Yes', 'No', 'Yes', 'No', 'No', 'Yes', 'No', 'Yes', 'No', 'No', 'Yes', 'No', 'No', 'No', 'Yes', 'No', 'Yes', 'No', 'No', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'No', 'No', 'Yes', 'No']
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

## 10. ChemDFM few-shot with 4 random examples

In [13]:
few_shot_prompter = FewShotPrompter1(template=few_shot_prompt, use_canonicalize_smiles=True,
                                    sample_dataset=train_dataset, sample_mode='random', sample_num=4)

main(dataset=test_dataset,
     batch_size=batch_size,
     total_batches=total_batches,
     model=model,
     prompt_generator=few_shot_prompter.generate_prompt)

  1%|          | 1/148 [00:08<21:20,  8.71s/it]

['Yes'] [0.99999999999999]


  1%|▏         | 2/148 [00:18<22:53,  9.41s/it]

['Yes'] [0.99999999999999]


  2%|▏         | 3/148 [00:28<23:01,  9.53s/it]

...

['Yes'] [0.99999999999999]


 99%|█████████▉| 147/148 [23:03<00:08,  8.64s/it]

['Yes'] [0.99999999999999]


100%|██████████| 148/148 [23:11<00:00,  9.40s/it]

['No'] [0.0]
['Yes', 'Yes', 'No', 'No', 'No', 'No', 'Yes', 'No', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'No', 'Yes', 'No', 'No', 'No', 'Yes', 'No', 'No', 'No', 'No', 'Yes', 'No', 'No', 'No', 'No', 'Yes', 'No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'No', 'No', 'No', 'No', 'No', 'No', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'No', 'No', 'Yes', 'No', 'No', 'No', 'No', 'Yes', 'No', 'No', 'Yes', 'No', 'Yes', 'No', 'No', 'No', 'No', 'No', 'Yes', 'No', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'No', 'No', 'No', 'No', 'Yes', 'No', 'No', 'Yes', 'No', 'No', 'Yes', 'No', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'No', 'No', 'Yes', 'Yes', 'No', 'No', 'No', 'No', 'Yes', 'No', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'No']
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

## 11. ChemDFM few-shot with 2 random examples

In [14]:
few_shot_prompter = FewShotPrompter1(template=few_shot_prompt, use_canonicalize_smiles=True,
                                    sample_dataset=train_dataset, sample_mode='random', sample_num=2)

main(dataset=test_dataset,
     batch_size=batch_size,
     total_batches=total_batches,
     model=model,
     prompt_generator=few_shot_prompter.generate_prompt)

  1%|          | 1/148 [00:05<14:38,  5.97s/it]

['No'] [0.0]


  1%|▏         | 2/148 [00:11<14:12,  5.84s/it]

['Yes'] [0.99999999999999]


  2%|▏         | 3/148 [00:18<15:23,  6.37s/it]

...

['Yes'] [0.99999999999999]


 99%|█████████▉| 147/148 [17:16<00:06,  6.48s/it]

['Yes'] [0.99999999999999]


100%|██████████| 148/148 [17:23<00:00,  7.05s/it]

['Yes'] [0.99999999999999]
['No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'No', 'Yes', 'No', 'No', 'No', 'No', 'No', 'Yes', 'Yes', 'No', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'Yes', 'No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'No', 'No', 'No', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'No', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', '

## 12. ChemDFM zero-shot

In [18]:
zero_shot_prompt = \
    "You are an expert chemist, your task is to predict the property of molecule using your experienced chemical property prediction knowledge.\nPlease strictly follow the format, no other information can be provided. Given the SMILES string of a molecule, the task focuses on predicting molecular properties, specifically wether a molecule is Clinically-trail-Toxic(Yes) or Not Clinically-trail-toxic (No) based on the SMILES string representation of each molecule. The task is to predict the binary label for a given molecule, please answer with only 'Yes' or 'No'."

class ZeroShotPrompter1(BasePrompter):
    def generate_prompt(self, query_smiles):
        if self.use_canonicalize_smiles:
            query_smiles = self.canonicalize_smiles(query_smiles)

        prompt = f"{self.template}\nSMILES:{query_smiles}\nAnswer:"

        if self.verbose:
            print(prompt)

        return prompt

zero_shot_prompter = ZeroShotPrompter1(template=zero_shot_prompt, use_canonicalize_smiles=True)

main(dataset=test_dataset,
     batch_size=batch_size,
     total_batches=total_batches,
     model=model,
     prompt_generator=zero_shot_prompter.generate_prompt)

  1%|          | 1/148 [00:04<11:14,  4.59s/it]

['No'] [0.0]


  1%|▏         | 2/148 [00:10<12:43,  5.23s/it]

['No'] [0.0]


  2%|▏         | 3/148 [00:14<11:34,  4.79s/it]

...

['No'] [0.0]


 99%|█████████▉| 147/148 [11:19<00:04,  4.46s/it]

['No'] [0.0]


100%|██████████| 148/148 [11:23<00:00,  4.62s/it]

['No'] [0.0]
['No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'Yes', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'Yes', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'Yes', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'Yes', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'Yes', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'Yes', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'Yes', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'Yes', 'No', 'Yes', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'Yes', 'No', 'No', 'No', 'Yes', 'No', 'No', 'No', 'No']
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0

# TanimotoSimilarity

## MACCS

### ChemLLM few-shot with 2 examples

## RDK

## Morgan

In [3]:
def smiles2maccs_fp(smiles: str):
    return MACCSkeys.GenMACCSKeys(Chem.MolFromSmiles(smiles))

def smiles2rdk_fp(smiles: str):
    return Chem.RDKFingerprint(Chem.MolFromSmiles(smiles))

def smiles2morgan_fp(smiles: str):
    return AllChem.GetMorganFingerprint(Chem.MolFromSmiles(smiles), 2)

from rdkit.Chem.Scaffolds import MurckoScaffold
from rdkit.Chem import rdMolDescriptors
def get_scaffold_fp(x):
    mol = Chem.MolFromSmiles(x)
    scaffold = MurckoScaffold.GetScaffoldForMol(mol)
    scaffold_fp = rdMolDescriptors.GetMorganFingerprint(scaffold, 2)
    return scaffold_fp

In [4]:
def calculate_similarity(smiles1, smiles2):
    mol1 = Chem.MolFromSmiles(smiles1)
    mol2 = Chem.MolFromSmiles(smiles2)
    fp1 = FingerprintMols.FingerprintMol(mol1) # 默认类型的指纹
    fp2 = FingerprintMols.FingerprintMol(mol2)
    # return DataStructs.TanimotoSimilarity(fp1, fp2)
    return DataStructs.CosineSimilarity(fp1, fp2)
    # return DataStructs.DiceSimilarity(fp1, fp2)


In [5]:
def molecule_to_maccs(smiles: str):
    molecule = Chem.MolFromSmiles(smiles)
    return np.packbits(MACCSkeys.GenMACCSKeys(molecule))

In [3]:
calculate_similarity('NS(=O)(=O)Cc1noc2ccccc12', 'COc1ccccc1OCC(O)COC(N)=O')

0.5804433496611696

In [12]:
DataStructs.CosineSimilarity(smiles2morgan_fp('NS(=O)(=O)Cc1noc2ccccc12'), smiles2morgan_fp('COc1ccccc1OCC(O)COC(N)=O'))

[12:51:46] DEPRECATION WARNING: please use MorganGenerator
[12:51:46] DEPRECATION WARNING: please use MorganGenerator


ArgumentError: Python argument types in
    rdkit.DataStructs.cDataStructs.CosineSimilarity(UIntSparseIntVect, UIntSparseIntVect)
did not match C++ signature:
    CosineSimilarity(ExplicitBitVect bv1, std::string pkl, bool returnDistance=0)
    CosineSimilarity(SparseBitVect bv1, std::string pkl, bool returnDistance=0)
    CosineSimilarity(ExplicitBitVect bv1, ExplicitBitVect bv2, bool returnDistance=0)
    CosineSimilarity(SparseBitVect bv1, SparseBitVect bv2, bool returnDistance=0)

In [8]:
def similarity_sample_examples(query, keys):
    similarities = []
    for k in keys:
        similarities.append(calculate_similarity(query, k))
    return np.array(similarities)

In [17]:
keys = ['CS(=O)(=O)CCNCc1ccc(-c2ccc3ncnc(Nc4ccc(OCc5cccc(F)c5)c(Cl)c4)c3c2)o1',
        'CC[C@]1(O)C[C@H]2CN(CCc3c([nH]c4ccccc34)[C@@](C(=O)OC)(c3cc4c(cc3OC)N(C=O)[C@H]3[C@@](O)(C(=O)OC)[C@H](OC(C)=O)[C@]5(CC)C=CCN6CC[C@]43[C@@H]65)C2)C1.O=S(=O)(O)O',
        'COc1ccccc1OCC(O)COC(N)=O',
        'CC1=C(C(=O)[O-])N2C(=O)[C@@H](NC(=O)[C@H]([NH3+])C3=CCC=CC3)[C@H]2SC1',
        ]
query = 'NS(=O)(=O)Cc1noc2ccccc12'
a = similarity_sample_examples(query, keys)
print(a)
print(np.argsort(-a)[:2])
print([keys[i] for i in (np.argsort(-a)[:2])])

[0.42777156 0.44281525 0.39622642 0.40063762]
[1 0]
['CC[C@]1(O)C[C@H]2CN(CCc3c([nH]c4ccccc34)[C@@](C(=O)OC)(c3cc4c(cc3OC)N(C=O)[C@H]3[C@@](O)(C(=O)OC)[C@H](OC(C)=O)[C@]5(CC)C=CCN6CC[C@]43[C@@H]65)C2)C1.O=S(=O)(O)O', 'CS(=O)(=O)CCNCc1ccc(-c2ccc3ncnc(Nc4ccc(OCc5cccc(F)c5)c(Cl)c4)c3c2)o1']


In [43]:
np.argsort(a)[::-1][:2]

array([1, 0])

In [26]:
AllChem.GetMorganFingerprint(Chem.MolFromSmiles('NS(=O)(=O)Cc1noc2ccccc12'), 2)

[17:56:45] DEPRECATION WARNING: please use MorganGenerator
